In [2]:
import sys
sys.path.append('../')

from modules.utils.imports import *
from modules.binn.model_wrapper import model_wrapper
from modules.binn.build_binns import BINN
from modules.loaders.format_data import format_data
from modules.loaders.visualize_training_data import plot_animation

%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()
np.set_printoptions(threshold=sys.maxsize)

In [4]:
def animate_training_data(x_train, y_train, save=False, name='simulation'):
    
    x_np = x_train.numpy(force=True)
    y_np = y_train.numpy(force=True)
    all_train_np = np.column_stack((x_np, y_np))
    
    t_array = np.unique(all_train_np[:, 1])
       
    # Define function to update data
    def animate(i):
        frame = all_train_np[all_train_np[:, 1] == i, :]
        frame_sorted = np.array(sorted(frame, key=lambda a: a[0]))
        u_line.set_xdata(frame_sorted[:, 0])
        v_line.set_xdata(frame_sorted[:, 0])
        u_line.set_ydata(frame_sorted[:, 2])
        v_line.set_ydata(frame_sorted[:, 3])
        ax.set_title(f'T = {i}')
        
    # Plot animation
    fig, ax = plt.subplots()
    ax.set(ylim=(0, 12))
    first_frame = all_train_np[all_train_np[:, 1] == 0, :]
    first_frame_sorted = np.array(sorted(first_frame, key=lambda a: a[0]))
    u_line = ax.plot(first_frame_sorted[:, 0], first_frame_sorted[:, 2], color='orange', label='Cdc42T')[0]
    v_line = ax.plot(first_frame_sorted[:, 0], first_frame_sorted[:, 3], color='blue', label='Cdc42D')[0]

    anim = animation.FuncAnimation(fig, animate, frames=t_array, repeat=True)
    
    if save:
        writergif = animation.PillowWriter(fps=5)
        anim.save(f'{name}.gif', writer=writergif)

    return anim

In [5]:
# Get GPU
device = torch.device('cpu')

# Load in data
path = '/work/users/s/m/smyersn/elston/projects/kinetics_binns/data/'
file_name = 'spikes_data.npz'

# input: [x, t], output: [u, v]
inputs, output_u, output_v, shape_u, shape_v = format_data(path+file_name)
training_data = np.column_stack((inputs, output_u, output_v))

In [11]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

epsilon = 1
num_points = 10

# define new points to interpolate to and array to save interpolated data
x_interp = np.linspace(0, 10, num_points)
training_data_interp = np.zeros((0,4))

# reduce points in training data
for t in np.unique(training_data[:, 1]):
    time_step = training_data[training_data[:, 1] == t]
    u_interp = np.interp(x_interp, time_step[:, 0], time_step[:, 2])
    v_interp = np.interp(x_interp, time_step[:, 0], time_step[:, 3])
    time_step_interp = np.column_stack((x_interp, np.repeat(t, len(x_interp)), u_interp, v_interp))
    training_data_interp = np.row_stack((training_data_interp, time_step_interp))

# shuffle data
N = len(training_data_interp)
p = np.random.permutation(N)
training_data_shuffled = training_data_interp[p[:]]

# add noise to outputs
noise = np.random.normal(0, epsilon, training_data_shuffled[:, 2:].shape)
training_data_noise = training_data_shuffled
training_data_noise[:, 2:] = training_data_noise[:, 2:] + noise

# split into train/val and convert to torch
M = len(training_data_noise)
split = int(0.8*M)
x_train = to_torch(training_data_noise[:split, :2])
y_train = to_torch(training_data_noise[:split, 2:])
x_val = to_torch(training_data_noise[split:, :2])
y_val = to_torch(training_data_noise[split:, 2:])

In [12]:
anim = animate_training_data(to_torch(training_data_noise[:, :2]), to_torch(training_data_noise[:, 2:]), save=True, name=f'epsilon_{epsilon}_num_points_{num_points}')
anim